In [ ]:
from sctoolbox.utils.jupyter import bgcolor, _compare_version

# change the background of input cells
bgcolor("PowderBlue", select=[5, 9, 12, 15, 22, 26])

nb_name = "03_normalization_batch_correction.ipynb"

_compare_version(nb_name)

# 03 - Normalization and Batch effect correction
<hr style="border:2px solid black"> </hr>

## 1 - Description

Similar to quality control and filtering this step is aimed to prepare the data to facilitate better results in the following analysis steps. However, with normalization and batch effect correction the aim is to refine the data points in a way that

1. comparability between e.g. samples is enhanced
2. the influence of outliers is mitigated
3. variances introduced by technical or otherwise unwanted sources are omitted from the dataset.

Since this reduces the overall noise, the embedding and clustering steps in particular benefit from these adjustments.

________

## 2 - Setup

In [ ]:
import sctoolbox
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
import sctoolbox.utils as utils
import scanpy as sc
import pandas as pd

sctoolbox.settings.settings_from_config("config.yaml", key="03")

sc.set_figure_params(vector_friendly=True, dpi_save=600, scanpy=False)

________

## 3 - Load anndata
Loads the anndata.h5ad from the last notebook and provides a basic overview.

In [ ]:
adata = utils.adata.load_h5ad("anndata_2.h5ad")

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

In [ ]:
replace_cols = ["meta age_dec", "meta sex", "meta c19_severity"]

for col in replace_cols:
    # Prüfen, ob die Spalte existiert
    if col in adata.obs.columns:
        # Falls die Spalte eine Kategorie ist, erweitere die erlaubten Kategorien
        if pd.api.types.is_categorical_dtype(adata.obs[col]):
            adata.obs[col] = adata.obs[col].cat.add_categories(["Unknown"])
        
        # Nun können wir die NaN-Werte ersetzen
        adata.obs[col] = adata.obs[col].fillna("Unknown")
    else:
        print(f"Spalte '{col}' existiert nicht in adata.obs")

In [ ]:
with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

_________

## 4 - General input

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Choose normalization method
# TF-IDF: dimensionality is reduced by LSI
# Total: dimensionality is reduced by PCA 
norm_method = 'tfidf'  # can be 'tfidf' or 'total'

# Choose if highly variable features should be used
use_highly_variable = False # weil diese konnten in Notebook 2 noch nicht berechnet werden bzw. sind unnötig 

# Set number of neighbors
n_neighbors = 22 # 1% der originalen Daten (Patienten)

# UMAP related settings 
metacol = 'meta c19_severity'  # some meta-column of interest. See tables above.
n_features = 'n_features'  # column name for the number of features. See tables above.

# batch correction: If True, several batch correction methods will be performed,
# you can choose the best one after
batch_column = 'atac plate'

perform_batch_correction = True
batch_methods = ["harmony", "scanorama", "combat"] # "mnn", "scanorama", "combat" 
threads = 8

________________

In [ ]:
# Ensure that the batch column is of type category
adata.obs[batch_column] = adata.obs[batch_column].astype(str).astype("category")

____

## 5 - Normalization
<hr style="border:2px solid black"> </hr>

This section performs the selected normalization method followed by a dimension reduction. The counts for each cell are normalized so that all cells have the same number of counts after normalization. This removes imbalances in sequencing depth to make the cells comparable.

The normalization method can be either TF-IDF or total count normalization. Term frequency-inverse document frequencies (TF-IDF), initially adopted by search engines, scores each variable (here open chromatin region) by their importance. It compares the frequency of a variable within a cell against the global occurrence over all cells thus highlighting cell defining variables. On the other hand, total count normalization adjusts the total count of each cell so that all cells have the same total count after normalization. A method frequently used for single cell RNA data.

After normalization a dimension reduction is computed, which depending on the normalization is either latent semantic indexing (LSI) for TF-IDF or principal component analysis (PCA) for total count. However, while differing in details both reduction methods are analogous and thus the following steps are the same independent of the chosen method. Since TF-IDF has been shown to be particularly effective for ATAC-seq data, it is used here as the default method.  
**DOI: [10.1038/nature25981](https://doi.org/10.1038/nature25981)**

In [ ]:
# Save raw layer before normalization
adata.layers["raw"] = adata.X.copy()

In [ ]:
adata = tools.norm_correct.normalize_adata(adata, norm_method, use_highly_variable=use_highly_variable)

___________

## 6 - Dimension reduction and neighbor graph
<hr style="border:2px solid black"> </hr>
Another important property of our data is its high dimensionality. However, this complexity hinders in-depth analysis e.g. the identification of cell states. Thus, dimension reduction algorithms are applied to reduce complexity while simultaneously retaining patterns, a crucial step to enable embedding and clustering. In other words, noise is reduced by removing low variance components as well as components explaining technical or otherwise unwanted factors (e.g. number of active genes, cell cycle, etc.) which also has the benefit of reducing the computational demand. Here the applied algorithms are either Principal Component Analysis (PCA) or Latent Semantic Indexing (LSI). The applied dimension reduction depends on the chosen normalization with total count using PCA and TF-IDF using LSI. For convenience, results of both methods are called principal components (PCs) in the following analysis steps.

**DOI: [10.1038/nmeth.4346](https://doi.org/10.1038/nmeth.4346)**  
**DOI: [10.1038/nature25981](https://doi.org/10.1038/nature25981)**

The following heatmaps and barplots are intended to identify potentially unwanted PCs by showing the PCs in combination with available observations (cell-related metrics) and variables (feature-related metrics). In general, **selected PCs should avoid correlations with metrics**, but the importance of metrics and the stringency of thresholds depends on the experiment and the underlying questions, and therefore requires careful consideration by the analyst.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# number of PCs shown within the heatmap
n_pcs_heatmap = 20

____

In [ ]:
 # PCA correlations with obs variables 
_ = pl.embedding.plot_pca_correlation(
    adata,
    n_components=n_pcs_heatmap,
    which="obs",
    title="Correlation of .obs columns with PCA loadings",
    save="PCA_correlation_obs.pdf"
)

In [ ]:
 # PCA correlations with var variables
_ = pl.embedding.plot_pca_correlation(
    adata,
    n_components=n_pcs_heatmap,
    which="var",
    title="Correlation of .var columns with PCA loadings",
    save="PCA_correlation_var.pdf"
)

_________

### 6.1 - Choose a subset of PCs

In case the above plots showed undesired correlation this section can be used to subset the PCs. The proposed PC subset is displayed as a plot with darker bars representing the selected PCs. Based on the selected `filter_methods`, a vertical and horizontal threshold line is displayed. PCs are filtered if they are below the horizontal threshold (`corr_thresh`) or if they are to the right of the vertical threshold line (`perc_thresh`).

| Parameter | Description | Options |
|:---:|:---|:---|
| subset_pcs | Whether the PCs should be filtered. | `True` or `False` |
| corr_thresh | Highest absolute correlation that is allowed. Will take the maximum correlation for each PC as shown in the heatmap above. | Expects a value between `0-1`. |
| perc_thresh | Top percentile of PCs that should be kept. | A value between `0-100`%. |
| filter_methods | The PCs will be filtered based on the given methods. E.g. for "variance" and "correlation" PCs are filtered on values from both methods and the intersection is used as the final subset. | Any combination of `["variance", "cumulative variance", "correlation"]` |
| basis | Compute correlation based on observations (cells) or variables (genes). | Either `obs` or `var`. |
|ignore_cols| List of column names to ignore for correlation | `None` or a list of column names|

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Whether PCs should be filtered
subset_pcs = True

corr_thresh = 0.6  # PCs with an absolut correlation above this will be filtered
perc_thresh = 50  # Top percentile of PCs that should be kept
filter_methods = ['variance', 'correlation']  # propose PCs based on the provided methods
basis = 'obs'  # base correlation on obs or var
ignore_cols = []  # List of column names to ignore for correlation

________

In [ ]:
selected_pcs = tools.dim_reduction.propose_pcs(
    anndata=adata,
    how=filter_methods,
    corr_thresh=corr_thresh,
    perc_thresh=perc_thresh,
    corr_kwargs={'method': 'spearmanr', 'which': basis, 'ignore': ignore_cols}
)

# Plot and select number of PCs
_ = pl.embedding.plot_pca_variance(
    adata, 
    save='PCA_variance_proposed_selection.pdf',
    selected=selected_pcs,
    n_pcs=50,
    n_thresh=max(selected_pcs),
    corr_plot='spearmanr',
    corr_thresh=corr_thresh,
    corr_on=basis,
    ignore=ignore_cols
)

In [ ]:
f"Proposed principal components: {selected_pcs}"

Create a final PC-selection by changing the blue cell below:
- Either copy and adjust the proposed list from directly above
- create a custom list of PCs
- or accept the proposed list by not changing the cell below.

**Note: the selection will only be applied when `subset_pcs = True`.**

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
final_pc_selection = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] #selected_pcs

-------

In [ ]:
 _ = pl.embedding.plot_pca_variance(
    adata, 
    selected=final_pc_selection if subset_pcs else None,
    save='PCA_variance_final_selection.pdf',
    n_pcs=50,
    n_thresh=max(selected_pcs) if subset_pcs else None,
    corr_plot='spearmanr',
    corr_thresh=corr_thresh if subset_pcs else None,
    corr_on=basis,
    ignore=ignore_cols
)

In [ ]:
# Subset the number of pcs if chosen in the parameters
if subset_pcs:
    tools.dim_reduction.subset_PCA(adata, select=final_pc_selection)

___________

### 6.2 - Calculate neighbors
This step construct a graph connecting the cells with their k-nearest-neighbors based on the selected dimension reduction components. This graph represents the structure of the data and thus is used to detect clusters visualized in the UMAP in later steps.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=n_neighbors, method='umap', metric='euclidean')

________

## 7 - Batch correction
<hr style="border:2px solid black"> </hr>

Batch effects are variances in the data that are not intended by the experimental design (e.g. technical variance). They can be introduced through various sources. For example, sequencing samples at different timepoints may introduce batch effects. As batch effects could interfere with downstream analysis they are typically removed. However, it can be challenging to identify and correct for batch effects as this is highly dependent on the experimental setup of the dataset.

**DOI: [10.1038/nrg2825](https://doi.org/10.1038/nrg2825)**

There are several batch correction methods available, which may perform differently depending on the data set. Therefore, an overview is provided to compare batch correction methods and select the best performing one. To help in the decision making process, several metrics are shown that can be selected below and a score (LISI) is provided that explains whether the batches are well mixed after applying the correction.

In [ ]:
if perform_batch_correction:
    batch_corrections = tools.norm_correct.wrap_corrections(
        adata, 
        batch_key=batch_column,
        methods=batch_methods
    )
else:
    batch_corrections = {"uncorrected": adata}

__________

### 7.1 - Plot overview of batch corrections

In [ ]:
#Run standard umap for all adatas
tools.embedding.wrap_umap(batch_corrections.values(), threads=threads)

In [ ]:
default_embed_color = [k for k in adata.uns["sctoolbox"]["report"]["qc"]["obs"]["threshold"].keys() if k not in ["before", "after"]] + [batch_column]
default_embed_color

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Should preliminary clustering be performed?
do_clustering = True  # True or False

# embedding coloring
# choose the metrics shown in the following PCA and UMAP
# accepts adata.obs column names or genes (adata.var.index)
# if empty will use the list shown above
embed_color = ["meta age", "meta age_dec", "meta sex", "meta c19_severity", "atac reads aligned"]

In [ ]:
columns = [
    "atac plate",
    "meta age",
    "meta age_dec",
    "meta sex",
    "meta c19_severity",
    "atac reads aligned",
    "atac fraction of reads in peaks (frip)", # für WP3
    "n_fragments_wp3" # für WP3
]

for col in columns:
    unique_types = adata.obs[col].apply(type).unique()
    print(f"Datentypen in '{col}': {unique_types}\n")

_____________

In [ ]:
# Perform additional clustering if it was chosen
if do_clustering:
    for adata in batch_corrections.values():
        sc.tl.leiden(adata, resolution=0.1, flavor="igraph", n_iterations=2)
    (embed_color if embed_color else default_embed_color).append("leiden")
    
# Calculate LISI scores for batch
tools.norm_correct.wrap_batch_evaluation(batch_corrections, batch_key=batch_column, threads=threads, inplace=True)

**LISI score:**  
To determine the strength of a batch effect the Local Inverse Simpson's Index (LISI) can be used by measuring the heterogeneity within a local group. Comparing the LISI score between uncorrected data and the batch correction methods can help in deciding which method performed best.  
The LISI score (stored in `adata.obs`) indicates the effective number of different categories represented in the local neighborhood of each cell. If the cells are well-mixed, then we expect the LISI score to be closer to `n` for a dataset with `n` batches.

**DOI: [10.1038/s41592-019-0619-0](https://doi.org/10.1038/s41592-019-0619-0)**

**The higher the LISI score is, the better the batch correction method worked to normalize the batch effect and mix the cells from different batches.**

In [ ]:
# Plot the overview of batch correction methods
adata.obs[batch_column] = adata.obs[batch_column].astype("category")  # ensure that batch column is a category

_ = pl.embedding.anndata_overview(
    batch_corrections,
    color_by=embed_color if embed_color else default_embed_color,
    output=None
)

-----

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Choose an anndata object to proceed
selected = "harmony"

In [ ]:
adata_corrected = batch_corrections[selected]

_________

In [ ]:
if not perform_batch_correction and selected != "uncorrected":
    import warnings
    warnings.warn(f"Selected batch correction '{selected}' but batch correction is disabled. Falling back to 'uncorrected'.")
    
    selected = "uncorrected"
elif selected not in batch_corrections:
    raise KeyError(f"'{selected}' is not a key in batch_corrections. Choose one of: {list(batch_corrections.keys())}")

In [ ]:
adata = batch_corrections[selected]

______________

## 8 - Saving adata for the next notebook

In [ ]:
adata

In [ ]:
#Saving the data
adata_output = "anndata_3.h5ad"
utils.adata.save_h5ad(adata, adata_output)

In [ ]:
sctoolbox.settings.close_logfile()